## 3.softmax回归

**学习目标**

1. 熟悉softmax回归算法和模型构建方法

2. 熟练使用Softmax激活函数

3. 熟练使用交叉熵（CE）损失函数

4. 能够使用torchvision.datasets.MNIST加载MNIST数据集
****

3.1 softmax回归

逻辑回归主要用于二分类问题，即预测两个类别中的一个。它输出的是事件发生的概率。softmax回归可以看作是逻辑回归在多分类问题上的推广，它将一个样本的特征向量映射为概率分布。

Softmax回归用于多分类问题，能够处理两个以上类别的情况。它输出的是每个类别的概率。

3.2 MNIST数据集

MNIST数据集起源于美国国家标准与技术研究所（National Institute of Standards and Technology，简称NIST）。该数据集最初由NIST收集整理，由Yann LeCun等人在20世纪90年代创建，目的是通过算法实现对手写数字的识别。MNIST数据集是深度学习和图像识别研究中的一个基准数据集，包含了来自250个不同人的手写数字图片，其中一半是高中生的手写样本，另一半则来自人口普查局的工作人员。它包含了60,000个训练样本和10,000个测试样本，每个样本是一个28x28像素的灰度图像，代表从0到9的手写数字。

3.3 MNIST数据集的特点

简单性：由于图像都是标准化的尺寸和简单的手写数字，MNIST被认为是图像识别任务的一个相对简单的起点。

多样性：尽管每个数字只有10个样本，但手写风格的差异为模型提供了一定的挑战性。

广泛性：MNIST被广泛用于机器学习和深度学习算法的测试和训练，尤其是在卷积神经网络(CNN)的入门教程中。

基准测试：MNIST常被用作评估新算法性能的基准，因为它的数据量适中，且任务明确。

***

3.4 基于softmax回归的手写数字识别

1.导入必需的模块

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

2.数据集的加载和预处理

In [2]:
# 定义转换操作
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transforms.Compose 将transforms.ToTensor和transforms.Normalize等转换步骤组合起来，创建的预处理流程可以一次性应用到图像上。这使得数据预处理步骤的代码更加简洁和易于管理。

 在transforms.Normalize((0.5,), (0.5,)) 中：

第一个参数 (0.5,) 表示均值，它是一个单元素的元组，意味着所有通道的均值都是 0.5。由于只有一个元素，这通常用于灰度图像。

第二个参数 (0.5,) 表示标准差，它也是一个单元素的元组，意味着所有通道的标准差都是 0.5。

当你将这个变换应用到图像数据上时，每个通道的每个像素值将被减去均值 0.5 然后除以标准差 0.5。

这将导致变换后的像素值范围从原始的 [0, 1] 或 [0, 255] 变为 [-1, 1]。这种标准化有助于模型更快地收敛，因为它确保了不同通道和不同像素值的分布更加一致。

实际使用中，均值和标准差通常是根据整个数据集的统计数据计算得到的，以便更好地反映数据的分布特性。

In [3]:
# 下载训练集和测试集
train_dataset = datasets.MNIST(root='./datasets', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./datasets', train=False, download=True, transform=transform)

datasets.MNIST 是 PyTorch 的 torchvision.datasets 模块中的一个类，它用于加载和处理 MNIST 数据集。

使用 datasets.MNIST 加载数据集时，你可以指定数据集的根目录、转换（transforms）、下载方式等参数。这个类提供了一个方法 __getitem__ 来访问数据集中的每个样本，以及 __len__ 方法来获取数据集的大小。

使用 datasets.MNIST 类加载了训练集和测试集，通过 download=True 参数自动下载数据集（如果本地没有的话）。

In [4]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

DataLoader的作用是将数据集封装用于后面的训练，我们使用DataLoader来加载训练集和验证集。

3.模型构建

（1）参数设置

In [5]:
input_dim = 28*28  # MNIST图像大小为28x28
num_classes = 10   # 总共有10个类别（0到9）
num_epochs = 10

（2）构建Softmax回归模型

In [7]:
model = nn.Sequential(nn.Linear(input_dim, num_classes))

（3）定义损失函数和优化器

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

nn.CrossEntropyLoss()在PyTorch中已经内置了softmax函数。这个损失函数通常用于多分类问题，它结合了softmax层和负对数似然损失（negative log likelihood loss），因此你不需要在损失函数之前手动添加softmax层。
nn.CrossEntropyLoss的输入要求是未归一化的分数（即模型的原始输出），它将这些分数通过softmax函数转换为概率分布，然后计算这些概率与目标类别之间的交叉熵损失。

Softmax函数是一种常用的激活函数，特别是在处理多分类问题时。它将一个向量或一组实数转换成概率分布，使得每个元素的值都在0到1之间，并且所有元素的和为1。这使得Softmax函数非常适合用作神经网络输出层的激活函数，用于预测多个类别的概率。

在多分类问题中，Softmax函数通常与交叉熵损失（Cross-Entropy Loss）一起使用，这有助于优化模型以正确分类数据。

4.模型训练

In [9]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0  # 初始化总损失累加器
    
    for data, labels in train_loader:
        
        outputs = model(data.view(-1, input_dim))
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()  # 累加损失值

    # 计算平均损失并打印
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.5f}')

Epoch [1/10], Loss: 0.60564
Epoch [2/10], Loss: 0.38594
Epoch [3/10], Loss: 0.35212
Epoch [4/10], Loss: 0.33493
Epoch [5/10], Loss: 0.32395
Epoch [6/10], Loss: 0.31573
Epoch [7/10], Loss: 0.31012
Epoch [8/10], Loss: 0.30493
Epoch [9/10], Loss: 0.30125
Epoch [10/10], Loss: 0.29816


5.模型测试

In [10]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for data, labels in test_loader:
        
        outputs = model(data.view(-1, input_dim))
        # print(outputs[0])
        _, predicted = torch.max(outputs, 1)  # _ 最大值，predicted 最大值索引
        # print(predicted[0], labels[0])
        # print(labels.shape, labels.size(0))
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

Accuracy: 91.6%
